In [ ]:
import polars as pl
import polars_ds as pds
# Requires version >= v0.5.1
print(pds.__version__)

In [ ]:
size = 100_000
df = pds.random_data(size=size, n_cols=0).select(
    pds.random(0.0, 1.0).alias("x1"),
    pds.random(0.0, 1.0).alias("x2"),
    pds.random(0.0, 1.0).alias("x3"),
    pds.random_int(0, 3).alias("categories"),
    id = pl.Series(values=list(range(size)))
).with_columns(
    pl.col("id").cast(pl.UInt32),
    y = pl.col("x1") * 0.5 + pl.col("x2") * 0.25 - pl.col("x3") * 0.15 + pds.random() * 0.0001,
)

In [ ]:
# %%timeit
df.select(
    pds.query_lstsq_report(
        "x1", "x2", "x3",
        target = "y",
        add_bias = True
    )
).unnest("y")

In [ ]:
import numpy as np

np.hstack((x, np.ones((x.shape[0], 1))))


In [ ]:
from polars_ds._polars_ds import pds_linear_regression


In [ ]:
%%timeit
pds_linear_regression(x, y, False, "normal", 0., 0.)

In [ ]:

df.select(
    pds.query_lstsq(
        "x1", "x2", "x3",
        target = "y",
        method = "l2",
        l2_reg = 0.1,
    )
)

In [ ]:
from sklearn.linear_model import Lasso, Ridge, LinearRegression
df_x = df.select("x1", "x2", "x3").to_pandas()
df_y = df.select("y").to_pandas()

In [ ]:
%%timeit
reg = Ridge(alpha = 0.1, fit_intercept=False)
reg.fit(df_x, df_y)

In [ ]:
df.select(
    pds.query_knn_ptwise(
        "x1", "x2", "x3",
        index = "id",
        dist = "l1",
        k = 2,
        epsilon = 0.1
    )
)